In [1]:
# Create training data for DeppCT

In [2]:
import pyspark

sc = pyspark.SparkContext()
sc

<SparkContext master=yarn appName=pyspark-shell>

In [10]:
def map_to_queries(i):
    import json
    i = json.loads(i)
    
    return (i['targetDocumentId'], [j['anchorText'] for j in i['anchorTextSample']])

def to_json(i):
    import json
    
    anchorText = []
    for j in i[1]:
        anchorText += j
    
    return json.dumps({
        'targetDocumentId': i[0],
        'anchorText': anchorText
    })

sc.textFile('ecir22-filtered-commoncrawl-2019-47-sample/*/*')\
    .map(map_to_queries)\
    .groupByKey()\
    .map(to_json)\
    .repartition(10)\
    .saveAsTextFile('ecir22-filtered-commoncrawl-2019-47-deepct-sample.jsonl')

In [3]:
import spacy
spacy.__version__

'3.1.2'

In [21]:
#!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.6 MB 4.9 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
# From https://github.com/grill-lab/trec-cast-tools/blob/master/src/main/python/passage_chunker.py
import re
import spacy
nlp = spacy.load("en_core_web_sm", exclude=["parser", "tagger", "ner", "attribute_ruler", "lemmatizer", "tok2vec"]) #->senter
nlp.enable_pipe("senter")

print(nlp.pipe_names)


class SpacyPassageChunker:
    def __init__(self):
        self.document = None
        self.sentences = None

    
    def sentence_tokenization(self, document):
        self.document = parse_doc(document)
        self.sentences = list(self.document.sents)

    
    def create_passages(self, passage_size = 250):

        passages = []
        content_length = len(self.sentences)
        sentences_word_count = [len([token for token in sentence]) for sentence in self.sentences]
        
        current_idx = 0
        current_passage_word_count = 0
        current_passage = ''
        sub_id = 0
        
        for i in range(content_length):
            if current_passage_word_count >= (passage_size * 0.67):
                passages.append({
                    "body": current_passage,
                    "id": sub_id
                })
                current_passage = ''
                current_passage_word_count = 0
                
                current_idx = i
                sub_id += 1
            
            current_passage += self.sentences[i].text + ' '
            current_passage_word_count += sentences_word_count[i]

        
        current_passage = ' '.join(sentence.text for sentence in self.sentences[current_idx:])
        passages.append({
            "body": current_passage,
            "id": sub_id
        })
        
        return passages
        
        
def parse_doc(document):
    return nlp(sanitize_document(document))

def sanitize_document(doc):
    sanitized = re.compile('<.*?>')
    return re.sub(sanitized, '', doc)

def tokens(doc):
    return set([i for i in parse_doc(doc) if not i.is_punct])

['senter']


In [5]:
print(tokens('Speed Reading 4 Kid: Kill Toenail Fungus Now!'))

{4, Kid, Speed, Reading, Toenail, Now, Fungus, Kill}


In [6]:
import ir_datasets

ms_marco_docs = ir_datasets.load('msmarco-document').docs_store()

def passages(doc):
    ret = [doc.title]
    tmp = SpacyPassageChunker()
    tmp.sentence_tokenization(doc.body)
    
    ret += [i['body'] for i in tmp.create_passages(80)]

    return [(i[0] +1, i[1]) for i in enumerate(ret)]

#passages('D1112106')

In [9]:
ids = []

with open(DIR + 'cc-2019-47-anchors.jsonl', 'r') as f:
    for l in tqdm(f):
        ids += [json.loads(l)['targetDocumentId']]
ids = set(ids)

1163273it [00:48, 24186.80it/s]


In [1]:
import multiprocessing

pool = multiprocessing.Pool(processes=20)

def ms_marco_docs_iter():
    import ir_datasets
    iter = ir_datasets.load('msmarco-document').docs_iter()
    for doc in iter:
        if doc.doc_id in ids:
            yield doc

tmp = pool.map(lambda j: (j.doc_id, passages(j)), ms_marco_docs_iter())
ms_marco_doc_to_passages = {i[0]: i[1] for i in tqdm(tmp)}

ms_marco_doc_to_passages['D1112106']

NameError: name 'ids' is not defined

In [1]:
ms_marco_doc_to_passages['D1112106']

NameError: name 'ms_marco_doc_to_passages' is not defined

In [ ]:

ms_marco_doc_to_passages_processed = pool.map(ms_marco_doc_to_passages.values(), lambda i: (i.doc_id, passages(i)))

In [8]:
import json
from tqdm import tqdm
DIR = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/deep-ct/'

with open(DIR + 'cc-2019-47-anchors.jsonl', 'r') as f, open(DIR + 'deep-ct-training-data.jsonl', 'w') as out:
    for l in tqdm(f):
        l = json.loads(l)
        anchors = l['anchorText']
        
        for passage in passages(l['targetDocumentId']):
            out.write(json.dumps({'doc': {'title': passage}, 'queries': anchors}) + '\n')

0it [00:00, ?it/s]


AttributeError: 'str' object has no attribute 'body'